In [1]:
import torch
import torchvision.transforms as T

import PIL

import os
import random
import cv2
import numpy as np

# from matplotlib import pyplot as plt 
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import copy
import json

/homes/e34960/anaconda3/envs/patchdiff/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [83]:
coco_data_folder = "/homes/e34960/SegmentAndComplete/adv_data/coco_random_patch_100/data"
coco_files = os.listdir(coco_data_folder)

In [84]:
annotations_file = "/project/trinity/datasets/APRICOT_ORIGINAL/Annotations/apricot_annotations_test.json"

with open(annotations_file, 'r') as file:
    data = json.load(file)

annotations_coco_91 = {}

for i in range(len(data['categories'])):
    annotations_coco_91[data['categories'][i]['id']] = data['categories'][i]['name']

In [89]:
def getRandomImageMasks(n=1, scale=1.0, shape=(512,512)):
    random_numbers = random.sample(range(0, len(coco_files)-1), n)

    adv_imgs = []
    orig_imgs = []
    masks = []
    all_bboxes = []
    all_labels = []
    for i in range(n):
        img_info = torch.load(os.path.join(coco_data_folder, coco_files[random_numbers[i]]))
        adv_img = np.squeeze(img_info['x_adv'])
        adv_img = (255.0 * adv_img).astype(np.uint8)
        h, w, _ = adv_img.shape

        # adv_img = cv2.resize(adv_img, (int(w * scale), int(h * scale)))
        # adv_img = cv2.resize(adv_img, shape)
        adv_imgs.append(adv_img)

        orig_img = np.squeeze(img_info['x'])
        orig_img = (255.0 * orig_img).astype(np.uint8)
        h, w, _ = orig_img.shape

        # orig_img = cv2.resize(orig_img, (int(w * scale), int(h * scale)))
        # orig_img = cv2.resize(orig_img, shape)
        orig_imgs.append(orig_img)

        # mask = np.squeeze(img_info['Mask'])
        # mask = (255.0 * mask).astype(np.uint8)

        # # mask = cv2.resize(mask, (int(w * scale), int(h * scale)))
        # mask = cv2.resize(mask, shape)

        # # Make a rectangular mask
        # nonzero_indices = np.nonzero(mask)
        # min_row = np.min(nonzero_indices[0])
        # max_row = np.max(nonzero_indices[0])
        # min_col = np.min(nonzero_indices[1])
        # max_col = np.max(nonzero_indices[1])
        # mask[min_row:max_row+1, min_col:max_col+1] = 255

        # masks.append(mask)


        bounding_boxes = img_info['y']['boxes'].cpu().detach().numpy()
        bboxes = np.reshape(bounding_boxes, (bounding_boxes.shape[1], bounding_boxes.shape[2]))        
        bboxes[:, 2] = (bboxes[:, 2] - bboxes[:, 0]).astype(int)
        bboxes[:, 3] = (bboxes[:, 3] - bboxes[:, 1]).astype(int)
        # bboxes[:, 0] = (img.shape[0] * bboxes[:, 0]).astype(int)
        # bboxes[:, 1] = (img.shape[1] * bboxes[:, 1]).astype(int)

        all_bboxes.append(bboxes)

        labels = img_info['y']['labels'].cpu().detach().numpy()
        labels = np.reshape(labels, (labels.shape[1]))
        all_labels.append(labels)

    return adv_imgs, orig_imgs, masks, all_bboxes, all_labels

In [90]:
num_images = 20
adv_imgs, orig_imgs, masks, all_bboxes, all_labels = getRandomImageMasks(n=num_images, scale=0.20, shape=(512,512))

In [91]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
adversarial_images = copy.deepcopy(adv_imgs)
adversarial_results = model(adversarial_images)

original_images = copy.deepcopy(orig_imgs)
original_results = model(original_images)

Using cache found in /homes/e34960/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-5 Python-3.9.16 torch-1.8.1+cu102 CUDA:0 (GeForce RTX 2080 Ti, 11019MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [2]:
for i in range(num_images):
    fig, axes = plt.subplots(1, 3, figsize=(12, 20))

    axes[0].imshow(original_images[i])
    axes[0].axis('off')
    axes[0].set_title('GT Bounding Boxes')

    for j in range(all_bboxes[i].shape[0]):
        if all_labels[i][j] != -10:
            rectangle = patches.Rectangle((all_bboxes[i][j][0], all_bboxes[i][j][1]), all_bboxes[i][j][2], all_bboxes[i][j][3], linewidth=1, edgecolor='r', facecolor='none')
            # cv2.putText(imgs[i], text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            axes[0].add_patch(rectangle)
            axes[0].text(all_bboxes[i][j][0], all_bboxes[i][j][1], annotations_coco_91[all_labels[i][j]], fontsize=8, bbox=dict(facecolor='black', alpha=0.8, pad=1), color='white')
    

    adversarial_bboxes = (adversarial_results.pandas().xyxy[i]).to_numpy()
    adversarial_bboxes[:, 2] = (adversarial_bboxes[:, 2] - adversarial_bboxes[:, 0]).astype(int)
    adversarial_bboxes[:, 3] = (adversarial_bboxes[:, 3] - adversarial_bboxes[:, 1]).astype(int)
    adversarial_bboxes[:, 0] = (adversarial_bboxes[:, 0]).astype(int)
    adversarial_bboxes[:, 1] = (adversarial_bboxes[:, 1]).astype(int)

    axes[1].imshow(adv_imgs[i])
    axes[1].axis('off')
    axes[1].set_title('Adversarial Bounding Boxes')
    for j in range(adversarial_bboxes.shape[0]):
        rectangle = patches.Rectangle((adversarial_bboxes[j][0], adversarial_bboxes[j][1]), adversarial_bboxes[j][2], adversarial_bboxes[j][3], linewidth=1, edgecolor='r', facecolor='none')
        axes[1].add_patch(rectangle)
        axes[1].text(adversarial_bboxes[j][0],  adversarial_bboxes[j][1]-10, "{}:{:.2f}".format(adversarial_bboxes[j][6], adversarial_bboxes[j][4]) , fontsize=8, bbox=dict(facecolor='black', alpha=0.8, pad=1), color='white')


    original_bboxes = (original_results.pandas().xyxy[i]).to_numpy()
    original_bboxes[:, 2] = (original_bboxes[:, 2] - original_bboxes[:, 0]).astype(int)
    original_bboxes[:, 3] = (original_bboxes[:, 3] - original_bboxes[:, 1]).astype(int)
    original_bboxes[:, 0] = (original_bboxes[:, 0]).astype(int)
    original_bboxes[:, 1] = (original_bboxes[:, 1]).astype(int)

    axes[2].imshow(orig_imgs[i])
    axes[2].axis('off')
    axes[2].set_title('Original Image Bounding Boxes')
    for j in range(original_bboxes.shape[0]):
        rectangle = patches.Rectangle((original_bboxes[j][0], original_bboxes[j][1]), original_bboxes[j][2], original_bboxes[j][3], linewidth=1, edgecolor='r', facecolor='none')
        axes[2].add_patch(rectangle)
        axes[2].text(original_bboxes[j][0],  original_bboxes[j][1]-10, "{}:{:.2f}".format(original_bboxes[j][6], original_bboxes[j][4]) , fontsize=8, bbox=dict(facecolor='black', alpha=0.8, pad=1), color='white')


    # print(imgs[i].shape)
    # print(masks[i].shape)
    # repainted_bboxes = (repainted_results.pandas().xyxy[i]).to_numpy()
    # repainted_bboxes[:, 2] = (repainted_bboxes[:, 2] - repainted_bboxes[:, 0]).astype(int)
    # repainted_bboxes[:, 3] = (repainted_bboxes[:, 3] - repainted_bboxes[:, 1]).astype(int)
    # repainted_bboxes[:, 0] = (repainted_bboxes[:, 0]).astype(int)
    # repainted_bboxes[:, 1] = (repainted_bboxes[:, 1]).astype(int)

    # axes[2].imshow(repainted_images[i])
    # axes[2].axis('off')
    # axes[2].set_title('Repainted Bounding Boxes')
    # for j in range(repainted_bboxes.shape[0]):
    #     rectangle = patches.Rectangle((repainted_bboxes[j][0], repainted_bboxes[j][1]), repainted_bboxes[j][2], repainted_bboxes[j][3], linewidth=1, edgecolor='r', facecolor='none')
    #     axes[2].add_patch(rectangle)
    #     axes[2].text(repainted_bboxes[j][0],  repainted_bboxes[j][1]-10, "{}:{:.2f}".format(repainted_bboxes[j][6], repainted_bboxes[j][4]) , fontsize=8, bbox=dict(facecolor='black', alpha=0.8, pad=1), color='white')


    plt.tight_layout()
    plt.show()
    display(fig)
    plt.close()